## Rotor iteration tool

In [98]:
# Import

import numpy as np
import matplotlib.pyplot as plt

In [99]:
# Define atmospheric constants
rho = 0.017 # kg/m^3
g = 3.71 # m/s^2
a = 233.1 # m/s
T = 213.15 # K
Re_min = 10000
Re_max = 50000
mu = 1.13e-5 # Ns/m^2

#Define design parameters
M_tip = 0.7 # Tip Mach number
N = 6 # Number of ROTORS
N_blades = 2 # Number of blades per rotor
T_W_max = 1.2 # Thrust margin
T_A_disk = 8.5 # Disk loading [N/m^2]

# Define aerodynamic constants for rotor
cl_mean = 0.66 # Mean lift coefficient
cd_mean = 0.03 # Mean drag coefficient
k_hover = 1.2 # induced/ideal power ratio
ct_sigma = 0.115 

# Define flight time and battery specific energy
T_flight = 1800 # seconds
E_bat = 250 # Wh/kg

# Define mass constants
# Note that these can be changed as needed

m_payload = 11.87
m_avionics = 4.05
m_comms = 2.9
m_struct = 29.75
m_solar = 4

# Define variable masses

m_bat = 14
m_motor = 3
m_rotors = 12.66

In [100]:
# Define relevant functions

def calculate_V_tip(M_tip, a):
    return M_tip * a

def required_params_per_rotor(m_drone, g, N_rotor, disk_loading, thrust_margin):
    W = m_drone * g
    T_req = W * thrust_margin / N_rotor
    A_disk = T_req / disk_loading
    r_disk = np.sqrt(A_disk / np.pi)

    return W, T_req, A_disk, r_disk

def calculate_A_blade(V_tip, ct_sigma, T_req, rho):
    return T_req / (rho * ct_sigma * V_tip**2)

def calculate_ct_and_solidity(A_disk, rho, V_tip, T_req, ct_sigma):
     ct = T_req / (A_disk * rho * V_tip**2)
     sigma = ct / ct_sigma
     return ct, sigma

def calculate_omega(V_tip, r_disk):
    return V_tip / r_disk

def calculate_power_per_rotor(k_hover, T_req, rho, A_disk, A_blade, V_tip, cd_mean):
    P = k_hover * T_req * np.sqrt((T_req)/(2*rho*A_disk)) + rho * A_blade * V_tip**3 * cd_mean * (1/8)
    return P

def calculate_power_total(P_rotor, N_rotor):
    return P_rotor * N_rotor

def calculate_total_energy(P_total, T_flight):
    return (P_total * T_flight) / 3600 # Convert to Wh

def calculate_battery_mass(E_total, E_specific):
    return E_total / E_specific

def calculate_motor_mass(P_total):
    return 0.432/1000 * P_total


In [101]:
# Iterate battery mass only
m_drone = m_payload + m_avionics + m_comms + m_struct + m_solar + m_bat + m_motor + m_rotors
m_drone_estimate = 0
i = 0

while np.abs(m_drone - m_drone_estimate) > 0.0001 and i < 1000:

    # perform calculations...
    W, T_req, A_disk, r_disk = required_params_per_rotor(m_drone, g, N, T_A_disk, T_W_max)
    V_tip = calculate_V_tip(M_tip, a)
    A_blade = calculate_A_blade(V_tip, ct_sigma, T_req, rho)
    ct, sigma = calculate_ct_and_solidity(A_disk, rho, V_tip, T_req, ct_sigma)
    omega = calculate_omega(V_tip, r_disk)
    P_rotor = calculate_power_per_rotor(k_hover, T_req, rho, A_disk, A_blade, V_tip, cd_mean)
    P_total = calculate_power_total(P_rotor, N)
    E_total = calculate_total_energy(P_total, T_flight)
    m_bat = calculate_battery_mass(E_total, E_bat)

    # update drone mass estimate
    m_drone_estimate = m_payload + m_avionics + m_comms + m_struct + m_solar + m_bat + m_motor + m_rotors
    i += 1